In [63]:
import os
from math import radians, cos, isnan
from dotenv import load_dotenv
from datetime import datetime
from copy import deepcopy
import numpy as np
import pandas as pd
from mongoengine import connect, NotUniqueError
from pymongo import errors
import pygeohash as gh
import localization as lx
from deepdiff import DeepDiff
from stalkr import generate_grid_points
from grindr_access.grindrUser import grindrUser
from db_models import scrapedProfileModel, profileModel, profileHistoryModel, aggregatedProfileModel, profileLocationModel

load_dotenv()

connect(
    # db=os.getenv('MONGO_DB'),
    db="grindr",
    host=os.getenv('MONGO_URL'),
    port=27017,
    username=os.getenv('MONGO_USR'),
    password=os.getenv('MONGO_PWD'),
    authentication_source="admin"
    )

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), authsource='admin', uuidrepresentation=3, driver=DriverInfo(name='MongoEngine', version='0.28.2', platform=None))

In [64]:
user = grindrUser()
mail = os.getenv('GRINDR_MAIL')
password = os.getenv('GRINDR_PASS')
krk_lat = 50.059185
krk_lon = 19.937809

user.login(mail, password)

In [65]:
center_lat, center_lon = krk_lat, krk_lon # Equator and Prime Meridian
side_m = 10000 # 10 km side length
accuracy_m = 1000 # 1000m
points_per_side = int(side_m/accuracy_m) # Generate 100 points

grid_points = generate_grid_points(center_lat, center_lon, side_m, points_per_side,jitter_m=200)

In [66]:
scraped_profiles = {}
localization_data = {}
batch_timestamp = int(datetime.now().timestamp() * 1000)
i=0
for anchor_point in grid_points:
    created = int(datetime.now().timestamp() * 1000)
    anchor_gh = gh.encode(anchor_point[0], anchor_point[1],12)
    actual_anchor_point = list(gh.decode(anchor_gh))
    # Get the profiles for the current point from grindr API
    profile_list = user.getProfiles(actual_anchor_point[0], actual_anchor_point[1])
    print(f"{i}/{len(grid_points)}\t Anchor_point: {actual_anchor_point}\t Anchor_gh: {anchor_gh}")
    for response in profile_list['items']:
        response_profile = response['data']
        prof = {}
        # Filter out upsell banners or other non-profiles
        profile_types = ["PartialProfileV1", "FullProfileV1"]
        is_profile = any(element in response_profile.get('@type') for element in profile_types)
        has_distance = response_profile.get('distanceMeters') is not None
        if is_profile and has_distance:
            #fill all the fields obtained from request
            prof['profileId'] = response_profile['profileId']
            prof['created'] = created
            prof['anchor_lat'] =  actual_anchor_point[0] # Add the lat and lon of the anchor point
            prof['anchor_lon'] =  actual_anchor_point[1]
            prof['anchor_gh'] = anchor_gh
            prof['batch_timestamp'] = batch_timestamp
            prof['distance_from_anchor'] = response_profile['distanceMeters']
            # Save the profile to the database
            scraped_prof = scrapedProfileModel(**prof)
            scraped_prof.save()
            # store locally for further processing
            if scraped_profiles.get(scraped_prof.profileId) is None:
                scraped_profiles[scraped_prof.profileId] = []
                localization_data[scraped_prof.profileId] = []

            #rename the profileType field to avoid conflicts with mongoengine
            if response_profile.get('@type'):
                response_profile['profileType'] =  response_profile.pop('@type')
            keys_to_remove = ['upsellItemType', 'distanceMeters']
            for key in keys_to_remove:
                response_profile.pop(key, None)

            scraped_profiles[scraped_prof.profileId].append(deepcopy(response_profile))
            localization_data[scraped_prof.profileId].append(prof)
    i+=1


0/100	 Anchor_point: [50.014598, 19.868869]	 Anchor_gh: u2yhs0uh4hyg
1/100	 Anchor_point: [50.012465, 19.883089]	 Anchor_gh: u2yhs2qrtx3b
2/100	 Anchor_point: [50.015619, 19.899845]	 Anchor_gh: u2yhsc4dh50j
3/100	 Anchor_point: [50.013227, 19.913013]	 Anchor_gh: u2yht0shnv4x
4/100	 Anchor_point: [50.013796, 19.928356]	 Anchor_gh: u2yht2xrkr75
5/100	 Anchor_point: [50.01328, 19.947126]	 Anchor_gh: u2yhtbth3xp3
6/100	 Anchor_point: [50.012707, 19.961719]	 Anchor_gh: u2yhw0x9ju23
7/100	 Anchor_point: [50.015146, 19.975898]	 Anchor_gh: u2yhw8fp24vr
8/100	 Anchor_point: [50.014055, 19.993323]	 Anchor_gh: u2yhwby8yptg
9/100	 Anchor_point: [50.014967, 20.009651]	 Anchor_gh: u2yhx2fw69sg
10/100	 Anchor_point: [50.024979, 19.865915]	 Anchor_gh: u2yhs4cbtk5c
11/100	 Anchor_point: [50.023148, 19.881564]	 Anchor_gh: u2yhs6mmfn8g
12/100	 Anchor_point: [50.025403, 19.898324]	 Anchor_gh: u2yhsfce0dfp
13/100	 Anchor_point: [50.023966, 19.913097]	 Anchor_gh: u2yht4s68z14
14/100	 Anchor_point: [50.02539

In [67]:
for profileId, profiles in scraped_profiles.items():
    # iterate over all profiles and merge all the data by updating te output dict
    merged_profile_dict = profileModel().to_mongo().to_dict()

    for profile_dict in profiles:
        merged_profile_dict.update(profile_dict)

    profile = profileModel.objects(profileId=profileId).first()
    aggregated_profile = aggregatedProfileModel.objects(profileId=profileId).first()
    # if profile does not exist, create it
    if not profile:
        merged_profile_dict['created'] = batch_timestamp
        merged_profile_dict['updated'] = batch_timestamp
        profile = profileModel(**merged_profile_dict)

        aggregated_profile = aggregatedProfileModel(**merged_profile_dict)
        profile.save()
        aggregated_profile.save()

    current_profile_dict =  {}
    if profile:
        current_profile_dict = profile.to_mongo().to_dict()
    # those fielsd are not received from the API, so we need to remove them from the current profile
    keys_to_remove = ['_id','created', 'updated', 'last_lat', 'last_lon', 'cover_photo']

    for key in keys_to_remove:
        current_profile_dict.pop(key, None)

    # compare the current profile with the merged one
    diff = DeepDiff(current_profile_dict, merged_profile_dict, ignore_order=True, verbose_level=2).to_dict()
    # if there is a difference, update the profile
    if diff != {}:
        print("Profile has changed: ", profileId)
        print(diff)
        profileHistory = profileHistoryModel(profileId=profileId, timestamp=batch_timestamp, diff=diff)
        try:
            profileHistory.save()
        except (NotUniqueError, errors.DuplicateKeyError):
                print("A document with the same unique index already exists. Ignoring.")

        # update the profile
        merged_profile_dict['updated'] = batch_timestamp
        profile.update(**merged_profile_dict)
        #aggregate the profile history
        new_dict = deepcopy(current_profile_dict)
        new_dict.update(merged_profile_dict)
        new_dict['updated'] = batch_timestamp
        aggregated_profile.update(**new_dict)

Profile has changed:  506963836
{'dictionary_item_added': {"root['created']": 1710373679709, "root['updated']": 1710373679709}}
Profile has changed:  425268984
{'values_changed': {"root['profileType']": {'new_value': 'CascadeItemData$FullProfileV1', 'old_value': 'CascadeItemData$PartialProfileV1'}}, 'iterable_item_added': {"root['photoMediaHashes'][1]": 'd34cbcf857f26da69b4f1d742a58c79377ddbc9a', "root['photoMediaHashes'][2]": '5f41fac36ce9d3a7d514087afa5af75c07964c3c', "root['photoMediaHashes'][3]": 'd7d04fb33f5f34f191698c3ce969eea6bf1d09d8'}}
Profile has changed:  188456814
{'dictionary_item_added': {"root['created']": 1710373679709, "root['updated']": 1710373679709}}
Profile has changed:  607939503
{'values_changed': {"root['lastOnline']": {'new_value': 1710373580000, 'old_value': 1710372640000}}}
Profile has changed:  453366185
{'dictionary_item_added': {"root['created']": 1710373679709, "root['updated']": 1710373679709}}
Profile has changed:  602258836
{'values_changed': {"root['l

In [68]:
def select_indices(distances):
    # Convert distances to numpy array for efficient operations
    distances = np.array(distances)

    # Get the indices of the smallest 20 distances
    closest_indices = distances.argsort()[:20]
    return closest_indices

def localize(ref_points,distances)->list:
    P=lx.Project(mode='Earth1',solver='LSE')
    if any(isnan(distance) for distance in distances):
        return
    selected_indexes = select_indices(distances)
    if len(selected_indexes) < 3:
        return
    for i in selected_indexes:
        P.add_anchor(f'anchore_{i}',ref_points[i])
    t,label=P.add_target(ID=123)
    for i in selected_indexes:
        t.add_measure(f'anchore_{i}',distances[i])
    # with suppress_print():
    P.solve()

    return [t.loc.x, t.loc.y]

def localizeProfile(profiles, max_distance=1000):
    localizations = {}
    # Convert the list of dictionaries to a pandas DataFrame
    profiles_df = pd.DataFrame(profiles)
    profiles_df = profiles_df[profiles_df['distance_from_anchor'] <= max_distance]
    if profiles_df.empty:
        return localizations, None
    profile_id = profiles_df['profileId'].iloc[0]
    batch_timestamp = profiles_df['batch_timestamp'].iloc[0]
    localizedProfile = None
    if "distance_from_anchor" in profiles_df.columns:
        ref_points = [[lat, lon] for lat, lon in zip(profiles_df['anchor_lat'].tolist(), profiles_df['anchor_lon'].tolist())]
        distances = profiles_df['distance_from_anchor'].tolist()
        estimated_position = localize(ref_points,distances)
        estimated_gh = gh.encode(estimated_position[0], estimated_position[1],12) if estimated_position else ""
        localizations[profile_id] = {
            "estimated_position": estimated_position,
            "estimated_gh": estimated_gh,
            "batch_timestamp": batch_timestamp,
            "ref_points": ref_points,
            "distances": profiles_df['distance_from_anchor'].tolist()
        }
        if estimated_position:
            #sometimes not all fields are properly set in the request. This makes sure, that all data are available in database
            loc_prof = {
                "lat": estimated_position[0],
                "lon": estimated_position[1],
                "geoHash": estimated_gh,
                "profileId": profile_id,
                "timestamp": int(datetime.now().timestamp() * 1000),
                "batch_timestamp": batch_timestamp
            }
            localizedProfile = profileLocationModel(**loc_prof) # Create a new LocatedProfileModel instance


    return localizations, localizedProfile

In [69]:
#multilateration results, localized profiles. profileId is the key
ml_results = {}
# dict with localized profiles (LocationHistoryModel) . profileId is the key
localized_profiles = {}
# iterate over all profiles and localize them
for profileId, scraped_profileList in localization_data.items():
    print(f"Localizing profile {profileId}...")
    new_locations, loc_profile = localizeProfile(scraped_profileList, max_distance=2*accuracy_m)
    if loc_profile:
        localized_profiles[profileId] = loc_profile
        try:
            loc_profile.save()  # Attempt to save the new LocationHistoryModel profile
        except NotUniqueError:
            print("A document with the same unique index already exists. Ignoring.")
    ml_results.update(new_locations)


Localizing profile 506963836...
LSE Geolocating...
Localizing profile 425268984...
Localizing profile 608689794...
LSE Geolocating...
Localizing profile 575764954...
LSE Geolocating...
Localizing profile 188456814...
LSE Geolocating...
Localizing profile 605626903...
LSE Geolocating...
Localizing profile 240203576...
LSE Geolocating...
Localizing profile 602890040...
LSE Geolocating...
Localizing profile 407024401...
LSE Geolocating...
Localizing profile 445430209...
LSE Geolocating...
Localizing profile 607939503...
LSE Geolocating...
Localizing profile 548323276...
LSE Geolocating...
Localizing profile 480798103...
LSE Geolocating...
Localizing profile 596512305...
LSE Geolocating...
Localizing profile 612282296...
LSE Geolocating...
Localizing profile 600362825...
LSE Geolocating...
Localizing profile 453366185...
Localizing profile 175974448...
LSE Geolocating...
Localizing profile 602258836...
LSE Geolocating...
Localizing profile 478007265...
LSE Geolocating...
Localizing profile